In [1]:
import time
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import matplotlib.image as mpimg
from collections import OrderedDict
import pandas as pd


from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET 
import pandas as pd
from skimage.transform import AffineTransform, warp
from skimage.transform import rotate as rotate_transform
from skimage.util import random_noise
from skimage.filters import gaussian
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle

class Transforms():
    def __init__(self):
        pass
    
    def rotate(self, image, params):

        angle = params['rotation_range'][0]
        angle = (random.uniform(0,1))*random.choice([-1,1])*angle
        transformation_matrix = torch.tensor([
            [+cos(radians(angle)), -sin(radians(angle))], 
            [+sin(radians(angle)), +cos(radians(angle))]
        ])

        image = rotate_transform(np.array(image), angle = angle, mode = 'edge')

        # PIL expects RGB images to be uint with ranges from 0 to 255 so we have to convert it to a type that PIL can excpect ie a uint from 0 to 255 
        return Image.fromarray((image * 255).astype(np.uint8))

    def translation(self, image,  params):
        image_shape = np.array(image).shape
        ty = random.uniform(params['height_shift_range'][0]*image_shape[0],          
                            params['height_shift_range'][1]*image_shape[0])
        tx = random.uniform(params['width_shift_range'][0]*image_shape[1],
                            params['width_shift_range'][1]*image_shape[1] )

        
        horizontal_shift =  tx*random.choice([-1,1])
        vertical_shift = ty*random.choice([-1,1])
        horizontal_shift_normalised = horizontal_shift/image_shape[1]
        vertical_shift_normalised =  vertical_shift/image_shape[0]

        transform = AffineTransform(translation=(-horizontal_shift,-vertical_shift))

        image = warp(np.array(image),transform,mode='edge')


  
        # PIL expects RGB images to be uint with ranges from 0 to 255 so we have to convert it to a type that PIL can excpect ie a uint from 0 to 255 
        return Image.fromarray((image * 255).astype(np.uint8))
        
    def resize(self, image, img_size):
        image = TF.resize(image, img_size)
        return image

    def zoom(self, image, params):

        img_shape = np.array(image).shape
        zoom = random.uniform(params['zoom_range'][0],params['zoom_range'][1])
        image = TF.resize(image,(int(img_shape[0]*zoom), int(img_shape[1]*zoom)) )
        scale_transform = torch.tensor([[zoom, 0], 
                                        [0, zoom]])

        
        return image

    def color_jitter(self, image):
        color_jitter = transforms.ColorJitter(brightness=0.3, 
                                              contrast=0.3,
                                              saturation=0.3, 
                                              hue=0.1)
        image = color_jitter(image)
        return image


    def __call__(self, image, params, image_size):

        # set checked image and landmark to landmark_ and image_ (this is for making sure we use the last checked tranformed instead of wrongly tranformed to do the following               # tranform)
        
        # -----------------------
        image_ = Image.fromarray(image.copy())

        # -----------------------

        # ZOOM
        image  = self.zoom(image_,  params)
        

        # RESIZE

        image = self.resize(image, (image_size, image_size))

        # ----------------------
        #image_, landmarks_ = self.color_jitter(image_, landmarks_)
        # ----------------------
        
        # ROTATE
        image = self.rotate(image,  params)


        # ----------------------

        image = image
        # ----------------------

        # TRANSLATION
        image= self.translation(image, params)

 
        
        image = TF.to_tensor(image)
        # the following tranform normalises each channel to have a mean at 0.5 and std of 0.5 / NOTE: NOT sure if this is theoreticlly better, should check this
        image = TF.normalize(image, [0.5], [0.5])
        return image

class LandmarksDataset():

    def __init__(self, transform=None,zoom = [1.0 - 0.03258157476873315, 1.0 + 0.03258157476873315], rotation = [22], height_shift= [0,0.03003200603616672], width_shift= [0,0.03003200603616672 ]):

        # targets 0
        filenames1 = os.listdir('D:/Tsetse fly Project/Data/Missing_landmarkwings_L/')
        #filenames2 = os.listdir('D:/Tsetse fly Project/Data/Missing_landmarkwings_R/')
        # targets 1
        filenames3 = os.listdir('C:/Users/dylan/Desktop/goodwingsv20-21/')
    
        self. tranform = transform
        self.zoom = zoom
        self.rotation = rotation
        self.height_shift = height_shift
        self.width_shift = width_shift
        self.image_filenames = []
        self.targets = []
        self.image_size = 244
        self.transform = transform
        self.image_dir = 'D:/Tsetse fly Project/Data/Missing_landmarkwings_L/'
        
        #self.image_dir2 = 'D:/Tsetse fly Project/Data/Missing_landmarkwings_R/'
        self.image_dir3 = 'C:/Users/dylan/Desktop/goodwingsv20-21/'
        self.TransF_ = True

       # ------------------- Append left wings data to dataset class ------------

        for filename in filenames1:
            self.image_filenames.append(os.path.join(self.image_dir, filename))
            self.targets.append(1)

            

        # ------------------ Append flipped right wings data to dataset class-----


        #for filename in filenames2[:]:
        #    self.targets.append(1)
        #    self.image_filenames.append(os.path.join(self.image_dir2, filename))

        #num = len(self.targets.copy())
        for filename in filenames3:
            self.targets.append(0)
            self.image_filenames.append(os.path.join(self.image_dir3, filename))



        # ----------------------

    def TransF(self):
        self.TransF_ = True
    def NoTransF(self):
        self.TransF_ = False
    def resize(self,size):
        self.image_size = size
    def set_params(self, zoom = [0.95, 0.105], rotation = [10], height_shift= [0,0.05], width_shift= [0,0.05]):
        self.zoom = zoom
        self.rotation = rotation
        self.height_shift = height_shift
        self.width_shift = width_shift
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        params = {'zoom_range': self.zoom, 'rotation_range':self.rotation, 'height_shift_range': self.height_shift, 'width_shift_range': self.width_shift }
        image_ = plt.imread(self.image_filenames[index])
        target = torch.tensor(self.targets[index])

        image = plt.imread(self.image_filenames[index])

        
        if self.transform and self.TransF_:
            
            image = self.transform(image_, params, self.image_size)

        else:
            img_shape = image.copy().shape
            image = Image.fromarray(image)
            image = TF.resize(image, (self.image_size,self.image_size))
       
            image = TF.to_tensor(image)
            # the following tranform normalises each channel to have a mean at 0.5 and std of 0.5 / NOTE: NOT sure if this is theoreticlly better, should check this
            image = TF.normalize(image, [0.5], [0.5])

        return image, target

DataSet = LandmarksDataset(Transforms())

In [2]:
class resnet18_(nn.Module):
    def __init__(self,num_classes=1):
        super().__init__()
        self.model_name='resnet18'
        self.model=models.resnet18(pretrained = True)
        self.model.conv1=nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=torch.sigmoid(self.model(x))
        return x

In [3]:
def accuracy(predictions, y):
    return(sum((predictions.round() == y)) / float(len(y))).item()

    # helper functions
import sys

def print_overwrite(step, total_step, loss, operation):
    sys.stdout.write('\r')
    if operation == 'train':
        sys.stdout.write("Train Steps: %d/%d  Loss: %.6f " % (step, total_step, loss))   
    else:
        sys.stdout.write("Valid Steps: %d/%d  Loss: %.6f " % (step, total_step, loss))
        
    sys.stdout.flush()

In [4]:
DataSet.TransF()
DataSet.resize(244)
dataset = DataSet
# split the dataset into validation and test sets
len_valid_test_set = int(0.2*len(dataset)) # 60% training, 20% validation, 20% testing

len_train_set = len(dataset) - len_valid_test_set*2

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_test_set))
print("The length of Valid set is {}".format(len_valid_test_set))

train_dataset , valid_dataset, test_dataset  = torch.utils.data.random_split(dataset , [len_train_set, len_valid_test_set, len_valid_test_set], generator=torch.Generator().manual_seed(42))

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=20, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=True)


The length of Train set is 842
The length of Valid set is 280
The length of Valid set is 280


In [6]:
# feature transfer learning (none of th weights are frozen)

results = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val':[], 'time': []}
network = resnet18_().cuda()

# TRAIN
torch.autograd.set_detect_anomaly(True)

criterion = nn.BCELoss()
optimizer = optim.Adam(network.parameters(), lr=0.0001)
loss_min = np.inf
num_epochs = 30

start_time = time.time()
for epoch in range(1,num_epochs+1):
    acc_train = 0
    acc_valid = 0
    running_acc = 0
    loss_train = 0
    loss_valid = 0
    running_loss = 0
    
    network.train()
    for step in range(1,len(train_loader)+1):

        images, targets = next(iter(train_loader))

        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda().float() 
    
        predictions = network(images).flatten()

        optimizer.zero_grad()
    
        
        # find the loss for the current step
    
        loss_train_step = criterion(predictions, targets)


        acc_train_step = accuracy(predictions, targets)

        
        # calculate the gradients

        loss_train_step.backward()
    
        
        # update the parameters

        optimizer.step()
        acc_train += acc_train_step
        loss_train += loss_train_step.item()

        running_acc  = acc_train/step
        running_loss = loss_train/step

        
        print_overwrite(step, len(train_loader), running_loss, 'train')
        
    network.eval() 
    with torch.no_grad():
        
        for step in range(1,len(valid_loader)+1):
            
            
            images, targets = next(iter(valid_loader))

            if torch.cuda.is_available():
                images = images.cuda()
                targets = targets.cuda().float()
        
            predictions = network(images).flatten()

            # find the loss for the current step
            loss_valid_step = criterion(predictions, targets)

            acc_valid_step = accuracy(predictions, targets)
   
            acc_valid += acc_valid_step
  
            running_acc = acc_valid/step
            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')
    
    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)
    acc_train /= len(train_loader)
    acc_valid /= len(valid_loader)
    
    
    print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    print('--------------------------------------------------')
    print('Epoch: {}  Train acc: {:.5f}  Valid acc: {:.5f}'.format(epoch, acc_train, acc_valid))
    print('--------------------------------------------------')
    results['loss_train'].append(loss_train)
    results['loss_val'].append(loss_valid)
    results['acc_train'].append(acc_train)
    results['acc_val'].append(acc_valid)

    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), 'C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/classifiers/models/model_resnet18_classifer_finetune.pth') 
        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))
results['time'].append(time.time()-start_time)
del(network)
del(images)
del(targets)
del(predictions)
torch.cuda.empty_cache()

import pickle
f = open("C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/classifiers/training_losses/model_resnet18_classifer_finetune_trainingdata.pkl","wb")
pickle.dump(results,f)
f.close()





Valid Steps: 14/14  Loss: 0.684294 
--------------------------------------------------
Epoch: 1  Train Loss: 0.3848  Valid Loss: 0.6843
--------------------------------------------------
Epoch: 1  Train acc: 0.81176  Valid acc: 0.55357
--------------------------------------------------

Minimum Validation Loss of 0.6843 at epoch 1/30
Model Saved

Valid Steps: 14/14  Loss: 0.392730 
--------------------------------------------------
Epoch: 2  Train Loss: 0.1009  Valid Loss: 0.3927
--------------------------------------------------
Epoch: 2  Train acc: 0.96000  Valid acc: 0.81786
--------------------------------------------------

Minimum Validation Loss of 0.3927 at epoch 2/30
Model Saved

Valid Steps: 14/14  Loss: 0.146286 
--------------------------------------------------
Epoch: 3  Train Loss: 0.0796  Valid Loss: 0.1463
--------------------------------------------------
Epoch: 3  Train acc: 0.97059  Valid acc: 0.95000
--------------------------------------------------

Minimum Valida

In [6]:
# feature transfer learning (none of th weights are frozen)

results = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val':[], 'time': []}
network = resnet18_().cuda()

# TRAIN
torch.autograd.set_detect_anomaly(True)

criterion = nn.BCELoss()
optimizer = optim.Adam(network.parameters(), lr=0.0001)
loss_min = np.inf
num_epochs = 30

start_time = time.time()
for epoch in range(1,num_epochs+1):
    acc_train = 0
    acc_valid = 0
    running_acc = 0
    loss_train = 0
    loss_valid = 0
    running_loss = 0
    
    network.train()
    for step in range(1,len(train_loader)+1):

        images, targets = next(iter(train_loader))

        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda().float() 
    
        predictions = network(images).flatten()

        optimizer.zero_grad()
    
        
        # find the loss for the current step
    
        loss_train_step = criterion(predictions, targets)


        acc_train_step = accuracy(predictions, targets)

        
        # calculate the gradients

        loss_train_step.backward()
    
        
        # update the parameters

        optimizer.step()
        acc_train += acc_train_step
        loss_train += loss_train_step.item()

        running_acc  = acc_train/step
        running_loss = loss_train/step

        
        print_overwrite(step, len(train_loader), running_loss, 'train')
        
    network.eval() 
    with torch.no_grad():
        
        for step in range(1,len(valid_loader)+1):
            
            
            images, targets = next(iter(valid_loader))

            if torch.cuda.is_available():
                images = images.cuda()
                targets = targets.cuda().float()
        
            predictions = network(images).flatten()

            # find the loss for the current step
            loss_valid_step = criterion(predictions, targets)

            acc_valid_step = accuracy(predictions, targets)
   
            acc_valid += acc_valid_step
  
            running_acc = acc_valid/step
            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')
    
    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)
    acc_train /= len(train_loader)
    acc_valid /= len(valid_loader)
    
    
    print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    print('--------------------------------------------------')
    print('Epoch: {}  Train acc: {:.5f}  Valid acc: {:.5f}'.format(epoch, acc_train, acc_valid))
    print('--------------------------------------------------')
    results['loss_train'].append(loss_train)
    results['loss_val'].append(loss_valid)
    results['acc_train'].append(acc_train)
    results['acc_val'].append(acc_valid)

    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), 'C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/models/model_resnet18_classifer_finetune2.pth') 
        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))
results['time'].append(time.time()-start_time)
del(network)
del(images)
del(targets)
del(predictions)
torch.cuda.empty_cache()

import pickle
f = open("model_resnet18_classifer_finetune_trainingdata2.pkl","wb")
pickle.dump(results,f)
f.close()





Valid Steps: 11/11  Loss: 0.633360 
--------------------------------------------------
Epoch: 1  Train Loss: 0.4295  Valid Loss: 0.6334
--------------------------------------------------
Epoch: 1  Train acc: 0.79714  Valid acc: 0.63182
--------------------------------------------------

Minimum Validation Loss of 0.6334 at epoch 1/30
Model Saved

Valid Steps: 11/11  Loss: 0.880317 
--------------------------------------------------
Epoch: 2  Train Loss: 0.1689  Valid Loss: 0.8803
--------------------------------------------------
Epoch: 2  Train acc: 0.93714  Valid acc: 0.56364
--------------------------------------------------
Valid Steps: 11/11  Loss: 0.274628 
--------------------------------------------------
Epoch: 3  Train Loss: 0.1079  Valid Loss: 0.2746
--------------------------------------------------
Epoch: 3  Train acc: 0.96143  Valid acc: 0.87727
--------------------------------------------------

Minimum Validation Loss of 0.2746 at epoch 3/30
Model Saved

Valid Steps: 11

In [6]:
import pickle
f = open("model_resnet18_classifer_finetune_trainingdata.pkl","wb")
pickle.dump(results,f)
f.close()


In [6]:
# feature transfer learning (none of th weights are frozen)

results = {'acc_train': [], 'acc_val': [], 'loss_train': [], 'loss_val':[], 'time': []}
network = resnet18_().cuda()
for param in network.model.parameters():
    param.requires_grad = False
in_features_fc = network.model.fc.in_features 
network.model.fc = nn.Linear(in_features_fc, out_features=1, bias=True)
# TRAIN
torch.autograd.set_detect_anomaly(True)

criterion = nn.BCELoss()
optimizer = optim.Adam(network.parameters())
loss_min = np.inf
num_epochs = 50

start_time = time.time()
for epoch in range(1,num_epochs+1):
    acc_train = 0
    acc_valid = 0
    running_acc = 0
    loss_train = 0
    loss_valid = 0
    running_loss = 0
    
    network.train()
    for step in range(1,len(train_loader)+1):

        images, targets = next(iter(train_loader))

        if torch.cuda.is_available():
            images = images.cuda()
            targets = targets.cuda().float() 
    
        predictions = network(images).flatten()

        optimizer.zero_grad()
    
        
        # find the loss for the current step
    
        loss_train_step = criterion(predictions, targets)


        acc_train_step = accuracy(predictions, targets)


        
        # calculate the gradients

        loss_train_step.backward()
    
        
        # update the parameters

        optimizer.step()
        acc_train += acc_train_step
        loss_train += loss_train_step.item()
        
        running_acc  = acc_train/step
        running_loss = loss_train/step

        
        print_overwrite(step, len(train_loader), running_loss, 'train')
        
    network.eval() 
    with torch.no_grad():
        
        for step in range(1,len(valid_loader)+1):
            
            
            images, targets = next(iter(valid_loader))

            if torch.cuda.is_available():
                images = images.cuda()
                targets = targets.cuda().float()
        
            predictions = network(images).flatten()

            # find the loss for the current step
            loss_valid_step = criterion(predictions, targets)

            acc_valid_step = accuracy(predictions, targets)

            acc_valid += acc_valid_step
            running_acc = acc_valid/step
            loss_valid += loss_valid_step.item()
            running_loss = loss_valid/step

            print_overwrite(step, len(valid_loader), running_loss, 'valid')
    
    loss_train /= len(train_loader)
    loss_valid /= len(valid_loader)
    acc_train /= len(train_loader)
    acc_valid /= len(train_loader)
    
    
    print('\n--------------------------------------------------')
    print('Epoch: {}  Train Loss: {:.4f}  Valid Loss: {:.4f}'.format(epoch, loss_train, loss_valid))
    print('--------------------------------------------------')
    results['loss_train'].append(loss_train)
    results['loss_train'].append(loss_valid)
    results['acc_train'].append(acc_train)
    results['acc_val'].append(acc_valid)

    if loss_valid < loss_min:
        loss_min = loss_valid
        torch.save(network.state_dict(), 'C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/models/model_resnet18_classifer_fixedfeatures.pth') 
        print("\nMinimum Validation Loss of {:.4f} at epoch {}/{}".format(loss_min, epoch, num_epochs))
        print('Model Saved\n')
print('Training Complete')
print("Total Elapsed Time : {} s".format(time.time()-start_time))
results['time'].append(time.time()-start_time)
del(network)
del(images)
del(targets)
del(predictions)
torch.cuda.empty_cache()

f = open("model_resnet18_classifer_fixedfeatures_trainingdata.pkl","wb")
pickle.dump(results,f)
f.close()







Train Steps: 2/14  Loss: 0.763188 

KeyboardInterrupt: 